In [0]:
import numpy as np
import pandas as pd
import string 
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
train_ = pd.read_csv('/content/drive/My Drive/train.csv', delimiter="\t")
test_=pd.read_csv('/content/drive/My Drive/test.csv', delimiter="\t")
print(train_.head())
print(test_.head())

            meta  uid sentiment
0           meta    3  negative
1              @    O       NaN
2  AdilNisarButt  Hin       NaN
3       pakistan  Hin       NaN
4             ka  Hin       NaN
              meta  uid sentiment
0             meta    8   neutral
1               RT  Eng       NaN
2                @    O       NaN
3  UAAPconfessions  Eng       NaN
4             Love  Eng       NaN


In [0]:
def extractdat(dataframe):
  data = dataframe.loc[:,['meta','sentiment']]
  t = ""
  x = list()
  y = list()
  for i in range(data.shape[0]):
    if(data.iloc[i,0]=="meta"):
      x.append(t.strip())
      y.append(str(data.iloc[i,1]))
      t=""
    else:
      t += (str(data.iloc[i,0])+" ")
  
  x.append(t.strip())
  
  return x[1:], y

In [0]:
X_train, Y_train = extractdat(train_)
X_test, Y_test = extractdat(test_)

In [65]:
print(len(X_train), len(Y_train), len(X_test), len(Y_test))

13524 13524 1869 1869


In [0]:
def clean(text, tknzr):
  tweet_tokens = tknzr.tokenize(text)
  t = ""
  for j in tweet_tokens:
    if(j not in string.punctuation and j not in ['…']):
      t += (j.lower())
      t+=" "
  return t.strip()

In [0]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [0]:
for i in range(len(X_train)):
  X_train[i]=(clean(X_train[i],tknzr))

In [0]:
for i in range(len(X_test)):
  X_test[i]=(clean(X_test[i], tknzr))

In [0]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(X_train)

In [0]:
X_train = tk.texts_to_sequences(X_train)
X_test = tk.texts_to_sequences(X_test)
tweets_lengths = list()

In [73]:
X_train = pad_sequences(X_train, maxlen=200, padding='post')
X_test = pad_sequences(X_test, maxlen=200, padding='post')
X_train = np.array(X_train, dtype='float32')
X_test = np.array(X_test, dtype='float32')
from keras.utils import to_categorical
def categorize(arr):
  cat = list()
  for i in arr:
    if(i == "positive"):
      cat.append(0)
    elif(i == "negative"):
      cat.append(1)
    else:
      cat.append(2)
  cat = np.array(cat,dtype='float32')
  categorical = to_categorical(cat)
  return categorical

Y_train = categorize(Y_train)
Y_test = categorize(Y_test)
print(len(Y_test))
print(len(X_test))
vocab_size = len(tk.word_index)
print(vocab_size)
embedding_weights = []
for char, i in tk.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)


1869
1869
880


In [74]:
from keras.models import Sequential
from keras.layers import Dense, Input, Conv1D, GlobalMaxPool1D, Embedding, Flatten, Dropout, MaxPooling1D
from keras.layers import LSTM, Lambda, TimeDistributed, Bidirectional, concatenate
embedding_size = vocab_size
conv_layers = [[256, 7, 3], [256, 7, 3], [256, 3, -1],[256, 3, -1], [256, 3, -1],[256, 3, 3]]
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length = 200, weights=[embedding_weights]))
for i, j, k in conv_layers:
    model.add(Conv1D(i, j, activation='relu'))
    if k != -1:
        model.add(MaxPooling1D(pool_size=k))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 880)          774400    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 194, 256)          1577216   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 58, 256)           459008    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 256)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 17, 256)           19686

In [75]:
print(len(X_test),len(Y_test))

1869 1869


In [76]:
print(len(X_train),len(Y_train))

13524 13524


In [77]:
model.fit(X_train, Y_train, batch_size=128, epochs=30, validation_data=(X_test, Y_test), verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13524 samples, validate on 1869 samples
Epoch 1/30
13524/13524 [==============================] - 9s 675us/step - loss: 1.0675 - acc: 0.4019 - val_loss: 1.0498 - val_acc: 0.4312
Epoch 2/30
13524/13524 [==============================] - 4s 316us/step - loss: 0.9852 - acc: 0.4873 - val_loss: 0.9995 - val_acc: 0.4799
Epoch 3/30
13524/13524 [==============================] - 4s 314us/step - loss: 0.9027 - acc: 0.5603 - val_loss: 1.0749 - val_acc: 0.4708
Epoch 4/30
13524/13524 [==============================] - 4s 315us/step - loss: 0.8396 - acc: 0.6060 - val_loss: 1.0115 - val_acc: 0.5099
Epoch 5/30
13524/13524 [==============================] - 4s 315us/step - loss: 0.7828 - acc: 0.6352 - val_loss: 1.0279 - val_acc: 0.5190
Epoch 6/30
13524/13524 [==============================] - 4s 314us/step - loss: 0.6990 - acc: 0.6942 - val_loss: 1.0970 - val_acc: 0.5302
Epoch 7/30
13524/13524 [===